# 문서 표현 (Document Representation)
## sklearn VS gensim
- Scikit-learn(Sklearn)은 텍스트 분류, 클러스터링, 피처 추출을 포함한 일반적인 머신 러닝 작업에 더 중점을 두고 있음. NLP 파이프라인에 쉽게 통합할 수 있는 다양한 잘 테스트되고 효율적인 머신 러닝 알고리즘을 제공. Sklearn은 텍스트 데이터에 고급 머신 러닝 기술을 적용해야 할 때 좋은 선택.

- Gensim은 주로 주제 모델링 및 단어 임베딩을 위해 설계되었습니다. 대규모 텍스트 코퍼스에서 Latent Dirichlet Allocation(LDA) 및 Word2Vec과 같은 비지도 모델을 훈련하고 적용하는 데 매우 효율적. Gensim은 대규모 텍스트 데이터로 작업하고 의미 있는 의미 표현을 추출해야 할 때 특히 유용.

  - 요약:
    - 텍스트 데이터에 고급 머신 러닝 알고리즘을 적용하기 위해 Sklearn을 사용합니다
    - 대규모 텍스트 컬렉션에서 주제 모델링 및 단어 임베딩 학습을 위해 Gensim을 사용합니다.

# 1.BoW (Bag of Words)
<img src="https://image.slidesharecdn.com/vector-space-models-170118145044/95/cs571-vector-space-models-3-638.jpg?cb=1485433004" />

https://en.wikipedia.org/wiki/Bag-of-words_model
https://www.slideshare.net/jchoi7s/cs571-vector-space-models

## 1.1. sklearn 활용


- CountVectorizer는 자동으로 토큰화를 해줌.
  - 공백, 일부 구두점은 자동으로 제거
  - 소문자 변환

In [130]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from IPython.core import display as ICD

docs = ['오늘 동물원에서 원숭이를 봤어',
        '오늘 동물원에서 코끼리를 봤어 봤어',
        '동물원에서 원숭이에게 바나나를 줬어 바나나를']

# CountVectorizer 객체 생성
count_vector = CountVectorizer()

# fit_transform : 주어진 문서에 대해 단어 집합을 학습하고, 인덱스에 맞춰 벡터화 진행
BoW = count_vector.fit_transform(docs)

# 희소 행렬(0인 부분은 제외하고 1인 부분의 좌표만 저장)
print(BoW) # (0,3) = 0번째 문서의 3번째 열에 1개 있음

  (0, 3)	1
  (0, 0)	1
  (0, 4)	1
  (0, 2)	1
  (1, 3)	1
  (1, 0)	1
  (1, 2)	2
  (1, 7)	1
  (2, 0)	1
  (2, 5)	1
  (2, 1)	2
  (2, 6)	1


In [128]:
# toarray()로 실제 array로 변환할 수 있음
print(BoW.toarray())

[[1 0 1 1 1 0 0 0]
 [1 0 2 1 0 0 0 1]
 [1 2 0 0 0 1 1 0]]


In [124]:
# get_feature_names_out()로 중복 제거(칼럼명)
vocab = count_vector.get_feature_names_out()
print(vocab)

['동물원에서' '바나나를' '봤어' '오늘' '원숭이를' '원숭이에게' '줬어' '코끼리를']


In [131]:
# BoW.toarray()에 빈도, vocab에 컬럼명
BoW_df = pd.DataFrame(BoW.toarray(), columns=vocab)
BoW_df

,동물원에서,바나나를,봤어,오늘,원숭이를,원숭이에게,줬어,코끼리를
0,1,0,1,1,1,0,0,0
1,1,0,2,1,0,0,0,1
2,1,2,0,0,0,1,1,0


In [127]:
for i in range(len(docs)):
    print("문서{} : {}".format(i+1,docs[i]))
    ICD.display(pd.DataFrame([BoW.toarray()[i]], columns=vocab))
    print("\n\n")

문서1 : 오늘 동물원에서 원숭이를 봤어


,동물원에서,바나나를,봤어,오늘,원숭이를,원숭이에게,줬어,코끼리를
0,1,0,1,1,1,0,0,0





문서2 : 오늘 동물원에서 코끼리를 봤어 봤어


,동물원에서,바나나를,봤어,오늘,원숭이를,원숭이에게,줬어,코끼리를
0,1,0,2,1,0,0,0,1





문서3 : 동물원에서 원숭이에게 바나나를 줬어 바나나를


,동물원에서,바나나를,봤어,오늘,원숭이를,원숭이에게,줬어,코끼리를
0,1,2,0,0,0,1,1,0


## 1.2. gensim 활용

- 토큰화 직접 해줘야함

In [132]:
import numpy as np
from gensim import corpora

docs = ['오늘 동물원에서 원숭이를 봤어',
        '오늘 동물원에서 코끼리를 봤어 봤어',
        '동물원에서 원숭이에게 바나나를 줬어 바나나를']

# 토큰화(단어 쪼개기)
doc_ls = [doc.split() for doc in docs]
doc_ls

[['오늘', '동물원에서', '원숭이를', '봤어'],
 ['오늘', '동물원에서', '코끼리를', '봤어', '봤어'],
 ['동물원에서', '원숭이에게', '바나나를', '줬어', '바나나를']]

- corpora.Dictionary()는 문서 리스트인 doc_ls를 받아서 문서 내의 모든 고유 단어를 추출하고 고유 ID를 부여 즉 어휘 사전이 만들어짐

In [134]:
# 사전 생성
id2word = corpora.Dictionary(doc_ls)
print(id2word)
print(dict(id2word))

Dictionary<8 unique tokens: ['동물원에서', '봤어', '오늘', '원숭이를', '코끼리를']...>
{0: '동물원에서', 1: '봤어', 2: '오늘', 3: '원숭이를', 4: '코끼리를', 5: '바나나를', 6: '원숭이에게', 7: '줬어'}


In [136]:
# id2word.doc2bow : 입력한 문서의 (인덱스, 등장 횟수) 반환
print(id2word.doc2bow(['동물원에서']))          # [(0,1)] = ['동물원에서']라는 문서에 0인덱스인 '동물원에서'가 하나 있다는 뜻
print(id2word.doc2bow(['봤어', '봤어','봤어'])) # [(1,3)] = ['봤어', '봤어','봤어']라는 문서에 1인덱스인 '봤어'가 세개 있다는 뜻

[(0, 1)]
[(1, 3)]


In [146]:
# 각 문서의 희소행렬 만들기
BoW = [id2word.doc2bow(doc) for doc in doc_ls]
print(BoW)

[[(0, 1), (1, 1), (2, 1), (3, 1)], [(0, 1), (1, 2), (2, 1), (4, 1)], [(0, 1), (5, 2), (6, 1), (7, 1)]]


In [138]:
# 칼럼 이름 뽑기
vocab = list(id2word.values())
print(vocab)

['동물원에서', '봤어', '오늘', '원숭이를', '코끼리를', '바나나를', '원숭이에게', '줬어']


sparse2full 함수의 역할: 희소 벡터로의 변환
- 입력: 희소 표현의 벡터와 전체 벡터의 길이.
  - BoW[i]: 희소 벡터 (예: 특정 위치에 값이 존재하는 리스트 또는 배열).
  - len(vocab): 전체 벡터의 길이 (이 벡터가 몇 개의 요소를 가져야 하는지).

In [157]:
# BoW[0] 문서의 빈도값
from gensim.matutils import sparse2full
sparse2full(BoW[0],len(vocab))

array([1., 1., 1., 1., 0., 0., 0., 0.], dtype=float32)

In [154]:
# 각 문서의 빈도 수
BoW_df = [sparse2full(BoW[i],len(vocab)) for i in range(len(doc_ls))]
Bow_df = pd.DataFrame(BoW_df, columns=vocab)
Bow_df

,동물원에서,봤어,오늘,원숭이를,코끼리를,바나나를,원숭이에게,줬어
0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
1,1.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0


In [153]:
for i in range(len(docs)):
    print("문서{} : {}".format(i+1,docs[i]))
    ICD.display(pd.DataFrame([sparse2full(BoW[i],len(vocab))], columns=vocab))
    print("\n\n")

문서1 : 오늘 동물원에서 원숭이를 봤어


,동물원에서,봤어,오늘,원숭이를,코끼리를,바나나를,원숭이에게,줬어
0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0





문서2 : 오늘 동물원에서 코끼리를 봤어 봤어


,동물원에서,봤어,오늘,원숭이를,코끼리를,바나나를,원숭이에게,줬어
0,1.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0





문서3 : 동물원에서 원숭이에게 바나나를 줬어 바나나를


,동물원에서,봤어,오늘,원숭이를,코끼리를,바나나를,원숭이에게,줬어
0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0


# 2.DTM (Document-Term Matrix)

## 2.1. sklearn 활용

In [48]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

docs = ['오늘 동물원에서 원숭이를 봤어',
        '오늘 동물원에서 코끼리를 봤어 봤어',
        '동물원에서 원숭이에게 바나나를 줬어 바나나를']

# 모델 불러오기
count_vect = CountVectorizer()

# 빈도 행렬 생성
DTM = count_vect.fit_transform(docs)
DTM.toarray()

array([[1, 0, 1, 1, 1, 0, 0, 0],
       [1, 0, 2, 1, 0, 0, 0, 1],
       [1, 2, 0, 0, 0, 1, 1, 0]])

In [49]:
# 문서 이름 생성(인덱스)
doc_names = ['문서'+str(i+1) for i in range(len(docs))]

# 단어 사전의 유니크 값 생성
vocab = count_vect.get_feature_names_out()

# DTM 행렬을 데이터프레임으로 변환
DTM_df = pd.DataFrame(DTM.toarray(), index=doc_names, columns=vocab)

# 결과 출력
DTM_df

,동물원에서,바나나를,봤어,오늘,원숭이를,원숭이에게,줬어,코끼리를
문서1,1,0,1,1,1,0,0,0
문서2,1,0,2,1,0,0,0,1
문서3,1,2,0,0,0,1,1,0


## 2.2. gensim 활용

In [158]:
import gensim
from gensim import corpora
import numpy as np
import pandas as pd

docs = ['오늘 동물원에서 원숭이를 봤어',
        '오늘 동물원에서 코끼리를 봤어 봤어',
        '동물원에서 원숭이에게 바나나를 줬어 바나나를']

# 1. 토큰화
doc_ls = [doc.split() for doc in docs]
doc_ls

[['오늘', '동물원에서', '원숭이를', '봤어'],
 ['오늘', '동물원에서', '코끼리를', '봤어', '봤어'],
 ['동물원에서', '원숭이에게', '바나나를', '줬어', '바나나를']]

In [53]:
# 2. Gensim의 사전을 생성
id2word = corpora.Dictionary(doc_ls)
print(dict(id2word))

{0: '동물원에서', 1: '봤어', 2: '오늘', 3: '원숭이를', 4: '코끼리를', 5: '바나나를', 6: '원숭이에게', 7: '줬어'}


In [159]:
# 각 문서별 빈도수 체크
id2word.doc2bow(doc_ls[0])

[(0, 1), (1, 1), (2, 1), (3, 1)]

In [56]:
# 3. 각 문서를 BoW 형식으로 변환
DTM = [id2word.doc2bow(doc) for doc in doc_ls]
print(DTM)

[[(0, 1), (1, 1), (2, 1), (3, 1)], [(0, 1), (1, 2), (2, 1), (4, 1)], [(0, 1), (5, 2), (6, 1), (7, 1)]]


### sparse2full VS corpus2dense
- sparse2full
  - Gensim의 sparse2full는 `단일 문서`의 BoW 벡터를 **희소 벡터**로 변환하는 데 사용.
  - 입력 데이터:  단일 문서의 BoW벡터(단어 ID, 등장 횟수)

    ```
    입력 데이터 : [(0, 1), (1, 1), (2, 1), (3, 1)]
    ```
  - 출력 데이터 : 빈도 수 1차원 배열
    ```
    출력 데이터 : array([1., 1., 1., 1., 0., 0., 0., 0.], dtype=float32)
    ```
- gensim.matutils.corpus2dense()
  - 역할: 희소 표현의 (BoW, copus)를 행렬로 변환 **희소 행렬(Dense Matrix)**로 변환.
  - 입력 데이터: 주로 Gensim의 corpus(BoW 형식으로 표현된 문서들의 리스트)를 사용.
  ```
  [[(0, 1), (1, 1), (2, 1), (3, 1)], [(0, 1), (1, 2), (2, 1), (4, 1)], [(0, 1), (5, 2), (6, 1), (7, 1)]]
  ```
  - 출력 데이터 : 희소 행렬(보통 전치해서 DTM으로 사용)
  ```
  array([[1., 1., 1.],
       [1., 2., 0.],
       [1., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 2.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)
```

In [57]:
# 행은 단어/ 열은 문서로 나옴
gensim.matutils.corpus2dense(DTM, num_terms=len(id2word))

array([[1., 1., 1.],
       [1., 2., 0.],
       [1., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 2.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [58]:
# 4. BoW 형식을 풀어서 DTM을 2D 행렬로 변환 num_terms는 총 단어 수(고유 단어)
DTM_matrix = gensim.matutils.corpus2dense(DTM, num_terms=len(id2word)).T
DTM_matrix

array([[1., 1., 1., 1., 0., 0., 0., 0.],
       [1., 2., 1., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 2., 1., 1.]], dtype=float32)

In [59]:
# 5. DataFrame으로 변환

# 문서 이름 생성(인덱스)
doc_names = ['문서'+str(i+1) for i in range(len(docs))]

df_DTM = pd.DataFrame(DTM_matrix, index=doc_names, columns=vocab)

# 결과 출력
ICD.display(df_DTM)

,동물원에서,바나나를,봤어,오늘,원숭이를,원숭이에게,줬어,코끼리를
문서1,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
문서2,1.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0
문서3,1.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0


# 3.TF-IDF (Term Frequency-Inverse Document Frequency)

<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/10109d0e60cc9d50a1ea2f189bac0ac29a030a00" />



*  TF(단어 빈도, Term Frequency) : 단어가 문서 내에 등장하는 빈도
*  IDF(역문서 빈도, Inverse Document Frequency) : 단어가 여러 문서에 공통적으로 등장하는 빈도
*  한 문서 내에 자주 등장하고 다른 문서에 자주 등장하지 않는 단어를 주요 단어로 판별할 수 있음

weighting schema|weight|설명
--|--|--
term frequency|<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/91699003abf4fe8bdf861bbce08e73e71acf5fd4" />|=토큰빈도/문서내토큰빈도
inverse document frequency|<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/864fcfdc0c16344c11509f724f1aa7081cf9f657" />|=log(총문서갯수/(토큰이 등장한 문서수))

# 1. IDF 공식 차이

두 라이브러리는 **IDF(Inverse Document Frequency)** 값을 계산할 때 약간의 차이가 있습니다. 이로 인해 최종 TF-IDF 값도 달라질 수 있습니다.

## Gensim에서의 IDF 공식:
Gensim은 다음과 같은 IDF 공식을 사용

$
IDF(t) = \log\left(\frac{N}{df(t)}\right)
$

- **N**: 전체 문서의 수
- **df(t)**: 단어 `t`가 등장한 문서의 수

즉, Gensim은 **문서에 등장한 단어의 빈도(df)**를 기준으로만 IDF 값을 계산.  


## Scikit-learn에서의 IDF 공식:
Scikit-learn에서는 IDF 값을 조금 다르게 계산:

$
IDF(t) = \log\left(\frac{N}{df(t)} + 1\right) + 1
$

- Scikit-learn은 **`+1`을 더하는 방식**으로 **IDF가 0이 되는 것을 방지**. 이는 특정 단어가 모든 문서에 등장할 때, IDF 값이 0이 되는 것을 막기 위한 조치.
- 또한, 마지막에 **`+1`을 추가**하여 결과값이 너무 낮아지지 않도록 함.


## 3.1. sklearn 활용

- todense()와 toarray()는 동일한 숫자 데이터를 포함하고 있으며, 두 함수 모두 TF-IDF 결과를 행렬의 형태로 반환.

  - 차이점은 데이터의 형식에 있음. todense()는 numpy.matrix 객체를 반환하고, toarray()는 numpy.ndarray를 반환.

  - 대부분의 경우, 두 방법 모두 동일한 결과를 제공.   
  행렬 연산이 필요한 경우에는 todense()가 더 적합할 수 있고,    
  다양한 차원의 데이터를 다루고 싶다면 toarray()가 더 적합할 수 있음. (matrix 객체는 2차원으로 고정)

- tfidf 결과는 희소 행렬로 반환하고 (문서 번호, 단어 ID)의 형태로 나타남

In [161]:
from sklearn.feature_extraction.text import TfidfVectorizer

docs = ["The cat sat on my face I hate a cat",
        "The dog sat on my bed I love a dog"]

# TF-IDF 객체 생성
tfidf_vect = TfidfVectorizer()

# 희소표현, 중요도
tfidf = tfidf_vect.fit_transform(docs)

print(tfidf)

  (0, 9)	0.2511643891128359
  (0, 1)	0.7060055705947859
  (0, 8)	0.2511643891128359
  (0, 7)	0.2511643891128359
  (0, 6)	0.2511643891128359
  (0, 3)	0.35300278529739293
  (0, 4)	0.35300278529739293
  (1, 9)	0.2511643891128359
  (1, 8)	0.2511643891128359
  (1, 7)	0.2511643891128359
  (1, 6)	0.2511643891128359
  (1, 2)	0.7060055705947859
  (1, 0)	0.35300278529739293
  (1, 5)	0.35300278529739293


In [172]:
# 학습된 단어 사전 보는 법
vocab = tfidf_vect.vocabulary_
print(vocab)

{'the': 9, 'cat': 1, 'sat': 8, 'on': 7, 'my': 6, 'face': 3, 'hate': 4, 'dog': 2, 'bed': 0, 'love': 5}


In [170]:
# 칼럼 이름 가져오기
tf_idf_columns = tfidf_vect.get_feature_names_out()
tf_idf_columns

array(['bed', 'cat', 'dog', 'face', 'hate', 'love', 'my', 'on', 'sat',
       'the'], dtype=object)

In [163]:
# todense()로 희소행렬 만드는 법
tf_idf_dense = tfidf.todense()
tf_idf_dense

matrix([[0.        , 0.70600557, 0.        , 0.35300279, 0.35300279,
         0.        , 0.25116439, 0.25116439, 0.25116439, 0.25116439],
        [0.35300279, 0.        , 0.70600557, 0.        , 0.        ,
         0.35300279, 0.25116439, 0.25116439, 0.25116439, 0.25116439]])

In [165]:
# todense()로 생성된 'numpy.matrix'는 형태를 변환시킬 수 없음(연산에 많이 씀)
todense = pd.DataFrame(data = tf_idf_dense, columns =tf_idf_columns)

display(todense)
print(type(tfidf.todense()))

,bed,cat,dog,face,hate,love,my,on,sat,the
0,0.000000,0.706006,0.000000,0.353003,0.353003,0.000000,0.251164,0.251164,0.251164,0.251164
1,0.353003,0.000000,0.706006,0.000000,0.000000,0.353003,0.251164,0.251164,0.251164,0.251164


<class 'numpy.matrix'>


In [166]:
# toarray()로 희소행렬 만드는 법
tf_idf_array = tfidf.toarray()
tf_idf_array

array([[0.        , 0.70600557, 0.        , 0.35300279, 0.35300279,
        0.        , 0.25116439, 0.25116439, 0.25116439, 0.25116439],
       [0.35300279, 0.        , 0.70600557, 0.        , 0.        ,
        0.35300279, 0.25116439, 0.25116439, 0.25116439, 0.25116439]])

In [168]:
# toarray()
toarray = pd.DataFrame(tf_idf_array, columns=tf_idf_columns)

display(toarray)
print(type(tfidf.toarray()))

,bed,cat,dog,face,hate,love,my,on,sat,the
0,0.000000,0.706006,0.000000,0.353003,0.353003,0.000000,0.251164,0.251164,0.251164,0.251164
1,0.353003,0.000000,0.706006,0.000000,0.000000,0.353003,0.251164,0.251164,0.251164,0.251164


<class 'numpy.ndarray'>


In [178]:
# tf-idf 벡터를 다시 문장으로 변환(완전 변환은 안 되는 듯)
original_docs = tfidf_vect.inverse_transform(tfidf)
print(original_docs)
print(f"doc1 : {' '.join(original_docs[0])}")
print(f"doc2 : {' '.join(original_docs[1])}")

[array(['the', 'cat', 'sat', 'on', 'my', 'face', 'hate'], dtype='<U4'), array(['the', 'sat', 'on', 'my', 'dog', 'bed', 'love'], dtype='<U4')]
doc1 : the cat sat on my face hate
doc2 : the sat on my dog bed love


## 3.2. gensim 활용

In [179]:
import gensim
from gensim import corpora
from gensim.models import TfidfModel
from gensim.matutils import corpus2dense
import pandas as pd
from IPython.core import display as ICD

docs = ["The cat sat on my face I hate a cat",
        "The dog sat on my bed I love a dog"]

# 토큰화
doc_ls = [doc.split() for doc in docs]
print(doc_ls)

[['The', 'cat', 'sat', 'on', 'my', 'face', 'I', 'hate', 'a', 'cat'], ['The', 'dog', 'sat', 'on', 'my', 'bed', 'I', 'love', 'a', 'dog']]


In [80]:
# 사전(Dictionary) 생성
id2words = corpora.Dictionary(doc_ls)
print(dict(id2words))

{0: 'I', 1: 'The', 2: 'a', 3: 'cat', 4: 'face', 5: 'hate', 6: 'my', 7: 'on', 8: 'sat', 9: 'bed', 10: 'dog', 11: 'love'}


In [81]:
# DTM 생성 (BoW 형식)
DTM = [id2words.doc2bow(doc) for doc in doc_ls ]
DTM

[[(0, 1), (1, 1), (2, 1), (3, 2), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)],
 [(0, 1), (1, 1), (2, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1)]]

In [85]:
# TF-IDF 모델 학습
model = TfidfModel(DTM)

# TF-IDF 값으로 DTM 행렬 생성
DTM_matrix = corpus2dense(model[DTM], num_terms=len(id2words)).T

# DataFrame으로 변환
df = pd.DataFrame(DTM_matrix, columns=id2words.values())

# 결과 출력
ICD.display(df)

,I,The,a,cat,face,hate,my,on,sat,bed,dog,love
0,0.0,0.0,0.0,0.816497,0.408248,0.408248,0.0,0.0,0.0,0.000000,0.000000,0.000000
1,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.408248,0.816497,0.408248


In [87]:
# (id, tf-idf값)
for doc in model[DTM]:
  print(doc)

[(3, 0.8164965809277261), (4, 0.4082482904638631), (5, 0.4082482904638631)]
[(9, 0.4082482904638631), (10, 0.8164965809277261), (11, 0.4082482904638631)]


In [88]:
model

In [89]:
list(model[DTM])

[[(3, 0.8164965809277261), (4, 0.4082482904638631), (5, 0.4082482904638631)],
 [(9, 0.4082482904638631), (10, 0.8164965809277261), (11, 0.4082482904638631)]]

In [90]:
list(model[DTM])[0]

[(3, 0.8164965809277261), (4, 0.4082482904638631), (5, 0.4082482904638631)]

In [91]:
list(model[DTM])[1]

[(9, 0.4082482904638631), (10, 0.8164965809277261), (11, 0.4082482904638631)]